In [3]:
import torch
import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader
import os 
from warnings import simplefilter
import pandas as pd
from imblearn.over_sampling import SMOTE  


import models
import class_sampling
import train
import metric_utils
import inference
import loss_fns
import torchvision.ops 

NUM_CLASSES = 10
n_epochs = 30
batch_size_train = 64
batch_size_test = 1000
momentum = 0

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

CLASS_LABELS = {'airplane': 0,
                 'automobile': 1,
                 'bird': 2,
                 'cat': 3,
                 'deer': 4,
                 'dog': 5,
                 'frog': 6,
                 'horse': 7,
                 'ship': 8,
                 'truck': 9}


simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=UserWarning)
simplefilter(action='ignore', category=DeprecationWarning)

col_names = ["name", 
            "num_classes", 
            "classes_used", 
            "ratio", 
            "learning_rate", 
            "mean_0", "variance_0",
            "mean_10", "variance_10",
            "mean_20", "variance_20",
            "mean_30", "variance_30",
          #   "mean_40", "variance_40",
          #   "mean_50", "variance_50",
             "cap", "normalization", "other"]

rows = []

In [4]:
# 3 classes

NUM_CLASSES_REDUCED = 3
nums = (0, 3, 1)
ratio = (200, 20, 1)

norm=False

if norm:
    transform=torchvision.transforms.Compose([torchvision.transforms.Normalize(mean=[134.1855, 122.7346, 118.3749], std=[70.5125, 64.4848, 66.5604])])
else:
    transform=None

    
train_CIFAR10 = torchvision.datasets.CIFAR10('cifar10', train=True, download=True,
                             transform=transform)  

test_CIFAR10 = torchvision.datasets.CIFAR10('cifar10', train=False, download=True,
                             transform=transform)  

train_CIFAR10.data = train_CIFAR10.data.reshape(50000, 3, 32, 32)
test_CIFAR10.data = test_CIFAR10.data.reshape(10000, 3, 32, 32)


reduced_train_CIFAR10 = class_sampling.Reduce(train_CIFAR10, NUM_CLASSES_REDUCED, nums=nums, CIFAR=True, transform=transform)
reduced_test_CIFAR10 = class_sampling.Reduce(test_CIFAR10, NUM_CLASSES_REDUCED, nums=nums, CIFAR=True, transform=transform)

ratio_train_CIFAR10 = class_sampling.Ratio(train_CIFAR10, NUM_CLASSES_REDUCED, ratio, nums=nums, transform=transform)
targets = ratio_train_CIFAR10.labels 
class_count = np.unique(targets, return_counts=True)[1]

smote_train_CIFAR10 = class_sampling.Smote(ratio_train_CIFAR10, 5000 * NUM_CLASSES_REDUCED, transform=transform)

triplet_train_CIFAR10 = class_sampling.ForTripletLoss(reduced_train_CIFAR10, smote=False, transform=transform, num_classes=3)
triplet_ratio_train_CIFAR10 = class_sampling.ForTripletLoss(ratio_train_CIFAR10, smote=False, transform=transform, num_classes=3)
triplet_smote_train_CIFAR10 = class_sampling.ForTripletLoss(smote_train_CIFAR10, smote=True, transform=transform, num_classes=3)

weight = 1. / class_count
samples_weight = weight[targets]
samples_weight = torch.from_numpy(samples_weight)
oversampler = torch.utils.data.WeightedRandomSampler(samples_weight, int(max(class_count) * NUM_CLASSES_REDUCED), replacement=True)
sampler = torch.utils.data.WeightedRandomSampler(samples_weight, len(samples_weight), replacement=True)
undersampler = torch.utils.data.WeightedRandomSampler(samples_weight, int(min(class_count) * NUM_CLASSES_REDUCED), replacement=False)

weight *= max(class_count)

train_loader_reduced = DataLoader(reduced_train_CIFAR10, batch_size=batch_size_train, shuffle=True)  

train_loader_ratio = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, shuffle=True) 

train_loader_oversampled = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, sampler=oversampler)

train_loader_undersampled = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, sampler=undersampler)

train_loader_sampled = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, sampler=sampler)

train_loader_smote = DataLoader(smote_train_CIFAR10, batch_size=batch_size_train, shuffle=True)

train_loader_tripletloss = DataLoader(triplet_train_CIFAR10, batch_size=batch_size_train, shuffle=True)

train_loader_tripletloss_ratio = DataLoader(triplet_ratio_train_CIFAR10, batch_size=batch_size_train, shuffle=True)

train_loader_tripletloss_smote = DataLoader(triplet_smote_train_CIFAR10, batch_size=batch_size_train, shuffle=True)

test_loader_reduced = DataLoader(reduced_test_CIFAR10, batch_size=batch_size_test, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# 3 class normal

learning_rates = [1e-4, 1e-3, 5e-4]

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_softmax(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_softmax(epoch, train_loader_reduced, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_softmax(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["normal", 3, nums, (1, 1, 1), learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], None, norm, None]
    rows.append(row)

In [14]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)
rows = [] 

In [ ]:
#  3 class ratio

learning_rates =  [1e-4, 1e-3, 5e-4]

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_softmax(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_softmax(epoch, train_loader_ratio, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_softmax(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["ratio", 3, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], None, norm, None]
    rows.append(row)

In [ ]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)
rows = [] 

In [3]:
# 3 class oversampled 

learning_rates = [1e-4, 5e-4, 1e-3]

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_softmax(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_softmax(epoch, train_loader_oversampled, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_softmax(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["oversampled", 3, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], None, norm, None]
    rows.append(row)


Test set: Avg. loss: 0.0033108042081197104, AUC: 0.5012746666666666


Test set: Avg. loss: 0.0010891745090484618, AUC: 0.5691605000000001


Test set: Avg. loss: 0.0010772271553675333, AUC: 0.6205455833333334


Test set: Avg. loss: 0.0010581616560618083, AUC: 0.66594325


Test set: Avg. loss: 0.009650338172912597, AUC: 0.64924925


Test set: Avg. loss: 0.0010989543994267782, AUC: 0.5006625


Test set: Avg. loss: 0.0010988156000773111, AUC: 0.5036341666666666


Test set: Avg. loss: 0.001098307967185974, AUC: 0.511603


Test set: Avg. loss: 0.0014643325408299763, AUC: 0.5624135


Test set: Avg. loss: 0.0010988226731618246, AUC: 0.5028216666666666


Test set: Avg. loss: 0.0010955985387166342, AUC: 0.5535775


Test set: Avg. loss: 0.0010860007206598917, AUC: 0.6126334166666666


Test set: Avg. loss: 0.0017339369455973306, AUC: 0.6287725000000001


Test set: Avg. loss: 0.0010934834877649943, AUC: 0.5562849166666667


Test set: Avg. loss: 0.0010735339721043905, AUC: 0.6279554166666667


Test

In [7]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)
rows = [] 

In [8]:
#  3 class SMOTE

learning_rates = [1e-4, 5e-4, 1e-3]

learning_rate_aucs = []

loss_fn_args = {}
loss_fn_args['loss_cap'] = None

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_softmax(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_softmax_with_smote(epoch, train_loader_smote, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_softmax(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["smote", 3, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], None, norm, None]
    rows.append(row)


Test set: Avg. loss: 0.0033108042081197104, AUC: 0.5012746666666666


Test set: Avg. loss: 0.0010981606642405193, AUC: 0.5078064999999999


Test set: Avg. loss: 0.0010916711489359539, AUC: 0.5696267500000001


Test set: Avg. loss: 0.0010836415688196819, AUC: 0.6216942499999999


Test set: Avg. loss: 0.001436800241470337, AUC: 0.4174843333333333


Test set: Avg. loss: 0.0010810662508010864, AUC: 0.6466885833333333


Test set: Avg. loss: 0.0010712401469548543, AUC: 0.6677557500000001


Test set: Avg. loss: 0.0010669514338175455, AUC: 0.6858748333333332


Test set: Avg. loss: 0.0074207375844319665, AUC: 0.47449125


Test set: Avg. loss: 0.00108564559618632, AUC: 0.5965848333333333


Test set: Avg. loss: 0.0010810999075571697, AUC: 0.6220325833333333


Test set: Avg. loss: 0.0010828034083048502, AUC: 0.6182540833333333


Test set: Avg. loss: 0.00392035977045695, AUC: 0.5374605


Test set: Avg. loss: 0.0010892696777979533, AUC: 0.6160569166666666


Test set: Avg. loss: 0.001085516333580017

In [9]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)
rows = [] 

In [10]:
# 3 class capped smote 


momentum=0
learning_rates = [5e-2]


cap_aucs = []

caps = [1, 5, 10]

for cap in caps:
    
    loss_fn_args = {}
    loss_fn_args['loss_cap'] = cap
    
    learning_rate_aucs = []

    for learning_rate in learning_rates:
        aucs = []
        for i in range(10):
            model_aucs = []
            network = models.ConvNet(NUM_CLASSES_REDUCED)
            optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
            _, auc = metric_utils.auc_softmax(test_loader_reduced, network) 
            model_aucs.append(auc)
            for epoch in range(n_epochs):
                _, _ = train.train_softmax_with_smote(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn=loss_fns.CappedCELoss, loss_fn_args=loss_fn_args)
                if (epoch + 1) % 10 == 0: 
                    _, auc = metric_utils.auc_softmax(test_loader_reduced, network)
                    model_aucs.append(auc)
            aucs.append(model_aucs)
        learning_rate_aucs.append(aucs)

    learning_rate_aucs = np.asarray(learning_rate_aucs)

    auc_mean = np.mean(learning_rate_aucs, axis=1)
    auc_variance = np.var(learning_rate_aucs, axis=1)
    
    
    cap_aucs.append([auc_mean, auc_variance])

    
    
for c in range(len(cap_aucs)):
    auc_mean = cap_aucs[c][0]
    auc_variance = cap_aucs[c][1]
    cap = caps[c]
    for i in range(len(learning_rates)): 
        row = ["capped_smote", NUM_CLASSES_REDUCED, nums, ratio, learning_rates[i],
                auc_mean[i][0], auc_variance[i][0], 
                auc_mean[i][1], auc_variance[i][1],
                auc_mean[i][2], auc_variance[i][2],
                auc_mean[i][3], auc_variance[i][3], cap, norm]
        rows.append(row)

print(rows)


Test set: Avg. loss: 0.004812867800394694, AUC: 0.5409411666666666


Test set: Avg. loss: 0.0023861385981241864, AUC: 0.5


Test set: Avg. loss: 0.002552573919296265, AUC: 0.5


Test set: Avg. loss: 0.0025949869950612388, AUC: 0.5


Test set: Avg. loss: 0.009052912712097167, AUC: 0.6076364166666667


Test set: Avg. loss: 0.002305452028910319, AUC: 0.5


Test set: Avg. loss: 0.002475385268529256, AUC: 0.5


Test set: Avg. loss: 0.0025396854877471923, AUC: 0.5


Test set: Avg. loss: 0.008161910692850749, AUC: 0.4842375833333333


Test set: Avg. loss: 0.0020253818829854327, AUC: 0.5001665


Test set: Avg. loss: 0.002539922555287679, AUC: 0.5001665


Test set: Avg. loss: 0.0026366058190663657, AUC: 0.4998333333333333


Test set: Avg. loss: 0.0038154197533925376, AUC: 0.6093339166666667


Test set: Avg. loss: 0.0023435554504394533, AUC: 0.5


Test set: Avg. loss: 0.0024914178053538003, AUC: 0.5


Test set: Avg. loss: 0.0025269065697987874, AUC: 0.5


Test set: Avg. loss: 0.0031156800587972

In [11]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)
rows = [] 

ValueError: 16 columns passed, passed data had 15 columns

In [48]:
# 3 class euclidean distance capped smote 


momentum=0
learning_rates = [1e-3, 5e-4, 1e-4]


cap_aucs = []

caps = [1, 5, 10]

loss_fn_args = {}
loss_fn_args['distance'] = 'euclidean'


for cap in caps:
    
    loss_fn_args['loss_cap'] = cap
    
    learning_rate_aucs = []

    for learning_rate in learning_rates:
        aucs = []
        for i in range(10):
            model_aucs = []
            network = models.ConvNetWithEmbeddings(NUM_CLASSES_REDUCED)
            optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
            _, auc = metric_utils.auc_softmax(test_loader_reduced, network, embeddings=True) 
            model_aucs.append(auc)
            for epoch in range(n_epochs):
                _, _ = train.train_softmax_with_embeddings(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn=loss_fns.CappedCELoss, loss_fn_args=loss_fn_args)
                if (epoch + 1) % 10 == 0: 
                    _, auc = metric_utils.auc_softmax(test_loader_reduced, network, embeddings=True)
                    model_aucs.append(auc)
            aucs.append(model_aucs)
        learning_rate_aucs.append(aucs)

    learning_rate_aucs = np.asarray(learning_rate_aucs)

    auc_mean = np.mean(learning_rate_aucs, axis=1)
    auc_variance = np.var(learning_rate_aucs, axis=1)
    
    
    cap_aucs.append([auc_mean, auc_variance])

    
    
for c in range(len(cap_aucs)):
    auc_mean = cap_aucs[c][0]
    auc_variance = cap_aucs[c][1]
    cap = caps[c]
    for i in range(len(learning_rates)): 
        row = ["distance_capped_smote", NUM_CLASSES_REDUCED, nums, ratio, learning_rates[i],
                auc_mean[i][0], auc_variance[i][0], 
                auc_mean[i][1], auc_variance[i][1],
                auc_mean[i][2], auc_variance[i][2],
                auc_mean[i][3], auc_variance[i][3], cap, norm]
        rows.append(row)

print(rows)



Test set: Avg. loss: 0.0053888258934020996, AUC: 0.5083743333333334


Test set: Avg. loss: 0.0011163065830866496, AUC: 0.5555713333333334


Test set: Avg. loss: 0.0011519194841384888, AUC: 0.6339973333333333


Test set: Avg. loss: 0.0012265472412109374, AUC: 0.6482106666666666


Test set: Avg. loss: 0.004785251299540202, AUC: 0.45894758333333335


Test set: Avg. loss: 0.0010515480041503906, AUC: 0.6808616666666666


Test set: Avg. loss: 0.001097139040629069, AUC: 0.6735478333333332


Test set: Avg. loss: 0.001158262848854065, AUC: 0.6645963333333333


Test set: Avg. loss: 0.002081297715504964, AUC: 0.5468329166666667


Test set: Avg. loss: 0.0010992099046707154, AUC: 0.6545398333333333


Test set: Avg. loss: 0.001181641697883606, AUC: 0.6227119999999999


Test set: Avg. loss: 0.0012087234656016032, AUC: 0.6747756666666667


Test set: Avg. loss: 0.0029571340878804526, AUC: 0.42823191666666666


Test set: Avg. loss: 0.0010765260457992554, AUC: 0.6793964166666666


Test set: Avg. loss: 0


Test set: Avg. loss: 0.0011079335610071819, AUC: 0.6403586666666666


Test set: Avg. loss: 0.0011695301135381062, AUC: 0.602681


Test set: Avg. loss: 0.0012256291309992473, AUC: 0.6285350833333333


Test set: Avg. loss: 0.005145518620808919, AUC: 0.44634475


Test set: Avg. loss: 0.0010701054732004801, AUC: 0.6592354166666666


Test set: Avg. loss: 0.0011435633500417074, AUC: 0.6172679166666667


Test set: Avg. loss: 0.0011791830857594808, AUC: 0.64276475


Test set: Avg. loss: 0.005740523020426432, AUC: 0.41456291666666667


Test set: Avg. loss: 0.0011070729494094848, AUC: 0.545332


Test set: Avg. loss: 0.0011548689603805542, AUC: 0.5669795000000001


Test set: Avg. loss: 0.001200473427772522, AUC: 0.6261900833333334


Test set: Avg. loss: 0.0022632593313852947, AUC: 0.4380240833333333


Test set: Avg. loss: 0.0010723592837651571, AUC: 0.6887599999999999


Test set: Avg. loss: 0.0011568965911865235, AUC: 0.6591684999999999


Test set: Avg. loss: 0.0012437734206517538, AUC: 0.604858


Test set: Avg. loss: 0.0011914784510930379, AUC: 0.6240734166666666


Test set: Avg. loss: 0.023554405212402343, AUC: 0.4700285


Test set: Avg. loss: 0.0010850404898325602, AUC: 0.6233955


Test set: Avg. loss: 0.0011104260285695394, AUC: 0.6535688333333333


Test set: Avg. loss: 0.0011573739051818847, AUC: 0.6540090833333333


Test set: Avg. loss: 0.00530566676457723, AUC: 0.4907729166666667


Test set: Avg. loss: 0.0011098883152008056, AUC: 0.5446788333333333


Test set: Avg. loss: 0.001145491639773051, AUC: 0.5830515833333333


Test set: Avg. loss: 0.0011320642630259195, AUC: 0.6684396666666665


Test set: Avg. loss: 0.0012915862401326498, AUC: 0.5365515


Test set: Avg. loss: 0.0011070706844329833, AUC: 0.5431425


Test set: Avg. loss: 0.0011233591238657633, AUC: 0.6475516666666667


Test set: Avg. loss: 0.0011704811652501425, AUC: 0.6520456666666666


Test set: Avg. loss: 0.009043893178304037, AUC: 0.3958669166666666


Test set: Avg. loss: 0.0010925205945968629, AUC: 0.606327833

In [49]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)
rows = [] 

In [ ]:
# 3 class cosine distance capped smote 


momentum=0
learning_rates = [1e-3, 5e-4, 1e-4]


cap_aucs = []

caps = [1, 5, 10]

loss_fn_args = {}
loss_fn_args['distance'] = 'cosine'


for cap in caps:
    
    loss_fn_args['loss_cap'] = cap
    
    learning_rate_aucs = []

    for learning_rate in learning_rates:
        aucs = []
        for i in range(10):
            model_aucs = []
            network = models.ConvNetWithEmbeddings(NUM_CLASSES_REDUCED)
            optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
            _, auc = metric_utils.auc_softmax(test_loader_reduced, network, embeddings=True) 
            model_aucs.append(auc)
            for epoch in range(n_epochs):
                _, _ = train.train_softmax_with_embeddings(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn=loss_fns.CappedCELoss, loss_fn_args=loss_fn_args)
                if (epoch + 1) % 10 == 0: 
                    _, auc = metric_utils.auc_softmax(test_loader_reduced, network, embeddings=True)
                    model_aucs.append(auc)
            aucs.append(model_aucs)
        learning_rate_aucs.append(aucs)

    learning_rate_aucs = np.asarray(learning_rate_aucs)

    auc_mean = np.mean(learning_rate_aucs, axis=1)
    auc_variance = np.var(learning_rate_aucs, axis=1)
    
    
    cap_aucs.append([auc_mean, auc_variance])

    
    
for c in range(len(cap_aucs)):
    auc_mean = cap_aucs[c][0]
    auc_variance = cap_aucs[c][1]
    cap = caps[c]
    for i in range(len(learning_rates)): 
        row = ["cosine_distance_capped_smote", NUM_CLASSES_REDUCED, nums, ratio, learning_rates[i],
                auc_mean[i][0], auc_variance[i][0], 
                auc_mean[i][1], auc_variance[i][1],
                auc_mean[i][2], auc_variance[i][2],
                auc_mean[i][3], auc_variance[i][3], cap, norm]
        rows.append(row)

print(rows)



Test set: Avg. loss: 0.0033108042081197104, AUC: 0.5012746666666666


Test set: Avg. loss: 0.0010315173864364625, AUC: 0.6993076666666668


Test set: Avg. loss: 0.001050257444381714, AUC: 0.7155940833333333


Test set: Avg. loss: 0.001184686779975891, AUC: 0.6812269999999999


Test set: Avg. loss: 0.001436800241470337, AUC: 0.4174843333333333


Test set: Avg. loss: 0.0010893462498982748, AUC: 0.6803560833333334


Test set: Avg. loss: 0.0010367254416147867, AUC: 0.7091099166666668


Test set: Avg. loss: 0.0011729620297749837, AUC: 0.7189865000000001


Test set: Avg. loss: 0.0074207375844319665, AUC: 0.47449125


Test set: Avg. loss: 0.001107176661491394, AUC: 0.5924970833333333


Test set: Avg. loss: 0.0011400168736775717, AUC: 0.6406568333333333


Test set: Avg. loss: 0.0012474682728449502, AUC: 0.6116938333333334


Test set: Avg. loss: 0.00392035977045695, AUC: 0.5374605


Test set: Avg. loss: 0.0011524693171183268, AUC: 0.6903105833333334


Test set: Avg. loss: 0.0011620090007781983

In [ ]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)
rows = [] 

In [ ]:
# 3 class triplet loss capped smote


momentum=0
learning_rates = [5e-2]

cap_aucs = []

start_epoch = 2

loss_caps = [1, 5, 10]
loss_fn_args = {}


for loss_cap in loss_caps:
    
    loss_fn_args['loss_cap'] = loss_cap
    
    learning_rate_aucs = []

    for learning_rate in learning_rates:
        aucs = []
        for i in range(10):
            model_aucs = []
            network = models.ConvNetWithEmbeddings(NUM_CLASSES_REDUCED)
            optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
            _, auc = metric_utils.auc_softmax(test_loader_reduced, network, embeddings=True) 
            model_aucs.append(auc)
            for epoch in range(start_epoch):
                loss_fn_args['loss_cap'] = None
                _, _ = train.train_softmax_with_embeddings(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn=loss_fns.CappedCELoss, loss_fn_args=loss_fn_args)
            for epoch in range(start_epoch, n_epochs + 1):
                loss_fn_args['loss_cap'] = loss_cap
                _, _ = train.train_triplet_capped_loss(epoch, train_loader_tripletloss_smote, network, optimizer, verbose=False, cap_calc=loss_fns.TripletLoss,loss_fn=loss_fns.CappedCELoss, loss_fn_args=loss_fn_args)
                if (epoch + 1) % 10 == 0: 
                    _, auc = metric_utils.auc_softmax(test_loader_reduced, network, embeddings=True)
                    model_aucs.append(auc)
            aucs.append(model_aucs)
        learning_rate_aucs.append(aucs)

    learning_rate_aucs = np.asarray(learning_rate_aucs)

    auc_mean = np.mean(learning_rate_aucs, axis=1)
    auc_variance = np.var(learning_rate_aucs, axis=1)
    
    
    cap_aucs.append([auc_mean, auc_variance])

    
    
for c in range(len(cap_aucs)):
    auc_mean = cap_aucs[c][0]
    auc_variance = cap_aucs[c][1]
    cap = loss_caps[c]
    for i in range(len(learning_rates)): 
        row = ["triplet_loss_capped_smote", NUM_CLASSES_REDUCED, nums, ratio, learning_rates[i],
                auc_mean[i][0], auc_variance[i][0], 
                auc_mean[i][1], auc_variance[i][1],
                auc_mean[i][2], auc_variance[i][2],
                auc_mean[i][3], auc_variance[i][3], cap, norm]
        rows.append(row)

print(rows)



Test set: Avg. loss: 0.0010990204016367595, AUC: 0.5136136666666666


Test set: Avg. loss: 0.0019028458992640176, AUC: 0.7512252500000001


Test set: Avg. loss: 0.002086396058400472, AUC: 0.7691906666666668


Test set: Avg. loss: 0.002416961034138997, AUC: 0.7808670000000001


Test set: Avg. loss: 0.0010924596786499024, AUC: 0.5750945000000001


Test set: Avg. loss: 0.001673059900601705, AUC: 0.7794949999999999


Test set: Avg. loss: 0.002390830198923747, AUC: 0.7884782499999999


Test set: Avg. loss: 0.002485786517461141, AUC: 0.7885251666666666


Test set: Avg. loss: 0.0010973639885584513, AUC: 0.5168527500000001


Test set: Avg. loss: 0.001288370688756307, AUC: 0.7993951666666667


Test set: Avg. loss: 0.001963376998901367, AUC: 0.8154784999999999


Test set: Avg. loss: 0.0018575411240259806, AUC: 0.8272758333333333


Test set: Avg. loss: 0.0011002889474232991, AUC: 0.5800430833333333


Test set: Avg. loss: 0.0014540077050526937, AUC: 0.7614485000000001


Test set: Avg. loss: 0.002

In [ ]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)
rows = [] 